In [500]:
import scipy.io
import numpy as np
from matplotlib import pyplot as plt
import glob
from scipy.signal import butter
import scipy
from hrvanalysis import remove_outliers, remove_ectopic_beats, interpolate_nan_values
from scipy import stats
import os
from scipy.sparse import spdiags

### Detrend signal

In [501]:
def detrend(signal, Lambda):
    signal_length = signal.shape[0]
    # observation matrix
    H = np.identity(signal_length)
    # second-order difference matrix
    ones = np.ones(signal_length)
    minus_twos = -2 * np.ones(signal_length)
    diags_data = np.array([ones, minus_twos, ones])
    diags_index = np.array([0, 1, 2])
    D = spdiags(diags_data, diags_index, (signal_length - 2), signal_length).toarray()
    filtered_signal = np.dot((H - np.linalg.inv(H + (Lambda ** 2) * np.dot(D.T, D))), signal)
    return filtered_signal


### Read mat files

In [502]:
onedrive_path = r'D:\OneDrive - UW\rPPG Clinical Study\UW Medicine Data'
all_mat_path = sorted(glob.glob(onedrive_path + "\ProcessedDataNoVideo\*.mat"))

### Remove artifacts

In [503]:
def remove_artifact(ppg):
    ppgmean = np.mean(ppg)
    ppgstddev = np.std(ppg[:200])
    f1 = np.where(ppg < ppgmean + 3 *ppgstddev)[0]
    f2 = np.where(ppg > ppgmean - 3 *ppgstddev)[0]
    f = np.intersect1d(f1, f2)
    return np.take(ppg, f)

### Filter

In [504]:
def filter_ppg(ppg, fs=60, lpf=0.7, hpf=4):    
    nyquistf = 1 / 2 * fs
    [b_pulse, a_pulse] = butter(3, [lpf / nyquistf, hpf/nyquistf], btype='bandpass')
    return scipy.signal.filtfilt(b_pulse, a_pulse, np.double(ppg))

## Normalization

In [505]:
def normalize_ppg(ppg):
    ppg = ppg - np.min(ppg)
    ppg = ppg / np.max(ppg)
    return ppg

### Find peaks

In [506]:
def find_peaks(ppg, height=0.6):
    peaks, _ = scipy.signal.find_peaks(ppg, height=height, distance=20)
    return peaks

### Remove ectopic beats

In [507]:
def remove_ectopic(ppg_peaks):
    rr_intervals_list = np.diff(ppg_peaks)
    # # This remove ectopic beats from signal
    nn_intervals_list = remove_ectopic_beats(rr_intervals=rr_intervals_list, method="malik")
    # This replace ectopic beats nan values with linear interpolation
    interpolated_nn_intervals = interpolate_nan_values(rr_intervals=nn_intervals_list)
    return np.cumsum(interpolated_nn_intervals).astype(np.int)

### Test

In [508]:
# mat = scipy.io.loadmat(all_mat_path[12])
# print(dir(mat))
# test_face_ppg = mat['ppg_face'][0]
# test_ppg = 1 - mat['ppg'][0]
# test_ppg = detrend(test_ppg, 100)
# test_ecg = mat['ekg'][0]
# cutoff = 3000
# test_face_ppg = normalize_ppg(filter_ppg(remove_artifact(test_face_ppg)))[:cutoff]
# test_ppg = normalize_ppg(filter_ppg(remove_artifact(test_ppg)))[:cutoff]
# test_ecg = normalize_ppg(test_ecg[:cutoff])
# print(test_ppg.shape)
# peaks_face_ppg = find_peaks(test_face_ppg)
# peaks_ppg = find_peaks(test_ppg)
# peaks_ecg = find_peaks(test_ecg)

# # peaks_face_ppg = remove_ectopic(peaks_face_ppg)
# # peaks_ppg = remove_ectopic(peaks_ppg)
# plt.figure()
# plt.plot(test_face_ppg)
# plt.plot(peaks_face_ppg, test_face_ppg[peaks_face_ppg], "x")
# plt.title('Face')
# plt.show()

# plt.figure()
# plt.plot(test_ppg)
# plt.plot(peaks_ppg, test_ppg[peaks_ppg], "x")
# plt.title('Finger')
# plt.show()

# plt.figure()
# plt.plot(test_ecg)
# plt.plot(peaks_ecg, test_ecg[peaks_ecg], "x")
# plt.title('ECG')
# plt.show()

### Calculate IBI

In [509]:
'''
1. detrend the finger ppg. 
2. filter and clean the finger ppg.
3. calculate ecg hr vs. finger ppg hr vs. face hr.
4. sync video using rr-interval. Max deday should be around 10s. 
'''

'\n1. detrend the finger ppg. \n2. filter and clean the finger ppg.\n3. calculate ecg hr vs. finger ppg hr vs. face hr.\n4. sync video using rr-interval. Max deday should be around 10s. \n'

In [510]:
def calcuate_window_ibi(finger_ppg, face_ppg, ecg, window_size=5, fs=60):
#     finger_ppg = detrend(finger_ppg, 100)
    face_ppg = normalize_ppg(filter_ppg(remove_artifact(face_ppg)))
    finger_ppg = normalize_ppg(filter_ppg(remove_artifact(finger_ppg)))
    ecg = normalize_ppg(ecg)
    finger_hr = []
    face_hr = []
    ecg_hr = []
    for idx in range(0, len(finger_ppg)-fs*window_size, fs*window_size):
        window_finger_ppg = finger_ppg[idx:idx + fs * window_size]
        window_face_ppg = face_ppg[idx:idx + fs * window_size]
        window_ecg = ecg[idx:idx + fs * window_size]
        window_peaks_finger_ppg = find_peaks(window_finger_ppg)
        window_peaks_face_ppg = find_peaks(window_face_ppg)
        window_peaks_ecg = find_peaks(window_ecg)
        window_avg_hr_finger = 60 / (np.mean(np.diff(window_peaks_finger_ppg)) / fs)
        window_avg_hr_face = 60 / (np.mean(np.diff(window_peaks_face_ppg)) / fs)
        window_avg_hr_ecg = 60 / (np.mean(np.diff(window_peaks_ecg)) / fs)
        finger_hr.append(window_avg_hr_finger)
        face_hr.append(window_avg_hr_face)
        ecg_hr.append(window_avg_hr_ecg)
    return finger_hr, face_hr, ecg_hr

def metrics_calculation(gt_hr, est_hr):
    gt_hr, est_hr = np.array(gt_hr), np.array(est_hr)
    mae = np.mean(np.abs(gt_hr - est_hr))
    rmse = np.sqrt(np.mean((gt_hr - est_hr)**2))
    mape = np.mean(np.abs(gt_hr - est_hr) / gt_hr)
#     pearson = stats.pearsonr(finger_hr, face_hr)[0]
    return mae, rmse, mape

In [511]:
# all_mat_path = [all_mat_path[2]]
finger_hr_all = []
face_hr_all = []
window_size = 60
for idx, mat_path in enumerate(all_mat_path):
    mat = scipy.io.loadmat(mat_path)
    face_ppg = mat['ppg_face'][0]
    finger_ppg = 1 - mat['ppg'][0]
    ecg = mat['ekg'][0]
    finger_hr, face_hr, ecg_hr = calcuate_window_ibi(finger_ppg, face_ppg, ecg, window_size=window_size, fs=60)
    mae, rmse, mape = metrics_calculation(ecg_hr, finger_hr)
#     mae, rmse, mape = metrics_calculation(ecg_hr, finger_hr)
#     finger_hr_all.extend(finger_hr)
#     face_hr_all.extend(face_hr)
    if mae > 5:
        print('mat_path: ', os.path.basename(mat_path))
        print('idx: ', idx)
        print(f'MAE: {mae}, RMSE: {rmse}, MAPE: {mape}')
# finger_hr_all, face_hr_all = np.array(finger_hr_all), np.array(face_hr_all)
# np.save(f'./finger_hr_all_ws_{str(window_size)}.npy', finger_hr_all)
# np.save(f'./face_hr_all_ws_{str(window_size)}.npy', face_hr_all)

mat_path:  P005a.mat
idx:  8
MAE: 6.119516666236358, RMSE: 6.119516666236358, MAPE: 0.11506783474974348
mat_path:  P005b.mat
idx:  9
MAE: 8.184694850412136, RMSE: 8.184694850412136, MAPE: 0.1603290780141844
mat_path:  P013b.mat
idx:  23
MAE: 63.68595945386535, RMSE: 63.68595945386535, MAPE: 0.49259252015134436
mat_path:  P015a.mat
idx:  26
MAE: 21.32118451025056, RMSE: 21.32118451025056, MAPE: 0.31044776119402967
mat_path:  P016a.mat
idx:  28
MAE: 46.272758800069354, RMSE: 46.272758800069354, MAPE: 0.6820052136577387
mat_path:  P017b.mat
idx:  31
MAE: 5.972831269436583, RMSE: 5.972831269436583, MAPE: 0.07817510517489913
mat_path:  P019a.mat
idx:  34
MAE: 72.08213297611209, RMSE: 72.08213297611209, MAPE: 0.5175462325398387
mat_path:  P019b.mat
idx:  35
MAE: 73.83857052517465, RMSE: 73.83857052517465, MAPE: 0.5213503482313356
mat_path:  P022b.mat
idx:  41
MAE: 6.111720240539043, RMSE: 6.111720240539043, MAPE: 0.0981047989430789
mat_path:  P026a.mat
idx:  48
MAE: 60.42987750771422, RMSE: 